In [1]:
from __future__ import print_function, division
from collections import Counter
import os
import time
import json
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme="chesterish")

src = "data"

In [80]:
# slice :100000 data for pilot study
# df = pd.read_csv('data/format1/user_log_format1.csv')
# df.iloc[:540000,:].to_csv('pilot0.1.csv', index=0)

# read pilot data
t0 = time.clock()
info = pd.read_csv('data/format1/user_info_format1.csv')
df_pilot = pd.read_csv('pilot.csv')
print("\nload data\nruntime: %.4f" % (time.clock() - t0))


load data
runtime: 0.3900


In [3]:
def get_purchase(df):
    '''
    :param : df, the raw dataframe
    :return: a dataframe, where the action type is 2
    >> select * from table where action_type=2
    '''
    return df[df.action_type == 2]


def get_distinct(df, by=['user_id', 'item_id']):
    '''
    :param : df, DataFrame
    :param : by, list, default = ['user_id', 'item_id']
    :return: a distinct Dataframe
    >> select distinct user_id, item_id from table
    '''
    return df.drop_duplicates(by)


def get_dataset(df, filtered):
    '''
    :param : df, the raw DataFrame
    :param : filtered, the distinct DataFrame
    :return: a Dataframe of list, 
             each DataFrame was corresponding to a record of the distinct DataFrame
    >> select * from table where user_id=???, item_id=???
    '''
    return map(lambda item: 
               df[(df.user_id == item[1][0]) & (df.item_id == item[1][1])],
               filtered.iterrows())


def get_action_count(df):
    '''
    :param : df, the distinct DataFrame
    :return: a count list of action_type, len=4, 
             where the index of list were corresponding to the action_type
    '''
    return map(lambda count: count[1], Counter(df.action_type).most_common())


def get_index(dataset):
    '''
    :param : dataset, a Dataframe of list, see also get_dataset()
    :return: pandas.Index
    '''
    for i, data in enumerate(dataset):
        index = data.index if i == 0 else index.append(data.index)
    return index

In [81]:
t0 = time.clock()
purchase_df = get_purchase(df_pilot)
print("\nget_purchase\nruntime:", time.clock() - t0)

t0 = time.clock()
purchase_distinct = get_distinct(purchase_df)
print("\nget_purchase_distinct\nruntime:", time.clock() - t0)

t0 = time.clock()
purchase_dataset = get_dataset(df_pilot, purchase_distinct)
print("\nget_purchase_dataset\nruntime:", time.clock() - t0)

t0 = time.clock()
index_purchase_dataset = get_index(purchase_dataset)
print("\nget_index\nruntime:", time.clock() - t0)

t0 = time.clock()
unpurchase_df = df_pilot.drop(index_purchase_dataset)
print("\nget_unpurchase\nruntime:", time.clock() - t0)

t0 = time.clock()
unpurchase_distinct = get_distinct(unpurchase_df)
print("\nget_unpurchase_distinct\nruntime:", time.clock() - t0)

t0 = time.clock()
unpurchase_dataset = get_dataset(unpurchase_df, unpurchase_distinct)
print("\nget_unpurchase_dataset\nruntime:", time.clock() - t0)


get_purchase
runtime: 0.00452009987202

get_purchase_distinct
runtime: 0.00747541138662

get_purchase_dataset
runtime: 22.6481534315

get_index
runtime: 1.21737449693

get_unpurchase
runtime: 0.0285363111598

get_unpurchase_distinct
runtime: 0.027373515084

get_unpurchase_dataset
runtime: 192.649772196


In [82]:
print("  dataset of purchase_distinct: %5d" % purchase_distinct.shape[0])
ct = sum(d.shape[0] for d in purchase_dataset)
print("              data in purchase: %5d" % ct)
print("dataset of unpurchase_distinct: %5d" % unpurchase_distinct.shape[0])
print("            data in unpurchase: %5d" % unpurchase_df.shape[0])

  dataset of purchase_distinct:  6177
              data in purchase: 26254
dataset of unpurchase_distinct: 52907
            data in unpurchase: 73746


In [89]:
x = purchase_dataset[0].groupby(['user_id', 'item_id', 'action_type'], as_index=0).count()
# df_pilot.groupby(['user_id', 'item_id', 'action_type'], as_index=0).count()
# pd.crosstab([df_pilot.user_id, df_pilot.item_id], df_pilot.action_type)  # fucking awesome
df = pd.read_csv('pilot.csv')
x = pd.crosstab([df.user_id, df.item_id], df.action_type)
pd.merge(purchase_dataset[1000], info, how='left', on='user_id')

In [2]:
t0 = time.clock()
df = pd.read_csv('data/format1/user_log_format1.csv')
print('load data\nruntime:', time.clock() - t0)

load data
runtime: 64.2329481168


In [4]:
t0 = time.clock()
crosstab_by_user_item_action = pd.crosstab([df.user_id, df.item_id], df.action_type)
print('runtime:', time.clock() - t0)

runtime: 414.164133998


In [18]:
# crosstab = crosstab_by_user_item_action.reset_index()
# crosstab_by_user_item_action.head()

In [34]:
x = np.array(crosstab_by_user_item_action.loc[:, [0, 1, 3]])
y = np.array(crosstab_by_user_item_action[2])
y[y > 0] = 1

In [37]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(tol=1e-8)

t0 = time.clock()
lr.fit(x, y)
print("runtime:", time.clock() - t0)

w = [lr.intercept_[0]] + [i for i in lr.coef_[0]]
print(w)

runtime: 124.45424361
[-3.0111185686109225, 0.4090775748229526, 0.15405716676367046, -0.2954124377754885]


In [ ]:
def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

nx_1 = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

t = nx_1.dot(w)
p = sigmoid(t)

plt.scatter(t, p)
plt.scatter(t, y, s=6)
# plt.axis([-10, 10, -.1, 1.1])
# np.where(t>40)
# nx[np.where(t>40)]

In [ ]:
y_hat = lr.predict(nx)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
# logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
# fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
print("roc_auc_score: ", roc_auc_score(ny, y_hat))
print("roc_auc_score: ", roc_auc_score(ny, lr.predict(nx)))

pred_prob = sigmoid(t) 

# fpr0, tpr0, thresholds0 = roc_curve(ny, pred_prob)
fpr1, tpr1, thresholds1 = roc_curve(ny, lr.predict_proba(nx)[:,1])

def roc_graph(fpr=fpr1, tpr=tpr1):
#     fig = plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % roc_auc_score(y, y_hat))
    plt.plot([0, 1], [0, 1],'w--')
    # plt.xlim([0.0, 1.0])
    # plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
#     plt.savefig('Log_ROC')
    plt.show()
#     return fig

roc_graph()

In [ ]:
conn = sqlite3.connect("skycat.db")

t0 = time.clock()
query = "select distinct user_id, item_id from user_log_batch where action_type=2"
cursor = conn.execute(query)
conn.commit()
result = cursor.fetchall()
print(len(result))
print("runtime: %.4f" % (time.clock() - t0))

In [ ]:
query = "select * from user_log_batch where user_id=? and item_id=?;"
for i, r in enumerate(result):
    cursor = conn.execute(query, r)
    conn.commit()
    subresult = cursor.fetchall()
    for sr in subresult:
        print(sr)
    if i == 0:
        break

In [ ]:
dst = "sliced_unique_purchase/"
buy_unique.iloc[0:1000000,:].to_csv(dst + 'p1.csv', index=False)
buy_unique.iloc[1000000:2000000,:].to_csv(dst + 'p2.csv', index=False)
buy_unique.iloc[2000000:2939895,:].to_csv(dst + 'p3.csv', index=False)
buy_unique.to_csv(dst + "all_purchase.csv", index=0)